# Интеллектуальный анализ текстов

In [1]:
import scipy.spatial
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import csr_matrix

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Расстояние Левенштейна 

In [2]:
fuzz.ratio("Игра", "Играть")/100

0.8

In [3]:
fuzz.ratio("Играющий", "Играть")/100

0.57

In [4]:
fuzz.ratio("Обедать", "Играть")/100

0.46

## Косинусное растояние
### Счетчик слов

In [5]:
sentences = [
    'привет друг',
    'привет привет',
    'как дела друг',
    'доброе утро',
    'что делаешь'
]
tokens = set(' '.join(sentences).split())
vectors = [[sentence.count(token) for token in tokens]
           for sentence in sentences]
print([(i, word) for i, word in enumerate(tokens)])
vectors

[(0, 'что'), (1, 'друг'), (2, 'привет'), (3, 'делаешь'), (4, 'как'), (5, 'доброе'), (6, 'утро'), (7, 'дела')]


[[0, 1, 1, 0, 0, 0, 0, 0],
 [0, 0, 2, 0, 0, 0, 0, 0],
 [0, 1, 0, 0, 1, 0, 0, 1],
 [0, 0, 0, 0, 0, 1, 1, 0],
 [1, 0, 0, 1, 0, 0, 0, 1]]

In [6]:
for i, sentence in enumerate(sentences):
    print(sentence)
    print(scipy.spatial.distance.cosine(vectors[0], vectors[i]))

привет друг
0.0
привет привет
0.29289321881345254
как дела друг
0.5917517095361369
доброе утро
1.0
что делаешь
1.0


### TF-IDF

In [7]:
vectorizer = TfidfVectorizer() #TfidfVectorizer(stop_words=['как', 'что'])
X = vectorizer.fit_transform(sentences)
len_vector = len(vectorizer.get_feature_names())
print([(i, word) for i, word in enumerate(vectorizer.get_feature_names())])
vectors = [list(i) for i in csr_matrix(X).toarray()]
vectors

[(0, 'дела'), (1, 'делаешь'), (2, 'доброе'), (3, 'друг'), (4, 'как'), (5, 'привет'), (6, 'утро'), (7, 'что')]


[[0.0, 0.0, 0.0, 0.7071067811865476, 0.0, 0.7071067811865476, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
 [0.6141889663426562,
  0.0,
  0.0,
  0.49552379079705033,
  0.6141889663426562,
  0.0,
  0.0,
  0.0],
 [0.0, 0.0, 0.7071067811865475, 0.0, 0.0, 0.0, 0.7071067811865475, 0.0],
 [0.0, 0.7071067811865475, 0.0, 0.0, 0.0, 0.0, 0.0, 0.7071067811865475]]

In [8]:
for i, sentence in enumerate(sentences):
    print(sentence)
    print(scipy.spatial.distance.cosine(vectors[0], vectors[i]))

привет друг
0.0
привет привет
0.2928932188134524
как дела друг
0.6496117672881416
доброе утро
1.0
что делаешь
1.0


In [9]:
new_sentence = 'как дела товарищ'
print(csr_matrix(vectorizer.transform([new_sentence])).toarray()[0])

[0.70710678 0.         0.         0.         0.70710678 0.
 0.         0.        ]


## По наиболее схожему вопросу найти ответ

In [14]:
df = pd.read_excel('sber_questions.xlsx', sheet_nam='All', names=['question', 'answer'])
df.head()

,question,answer
0,Как узнать условия кредитования?,"Узнайте подробные условия, по кредиту и рассчи..."
1,"Могу ли я взять кредит в Сбербанке, если у мен...","Да, вы можете получить в Сбербанке новый креди..."
2,Как подать заявку на кредит в Сбербанке?,Удобнее всего подать заявку на кредит в Сберба...
3,Как оформить кредит в Сбербанк Онлайн?,Чтобы подать заявку на кредит в Сбербанк Онлай...
4,Какой срок рассмотрения заявки на кредит в Сбе...,Максимальный срок рассмотрения заявки – 2 рабо...


In [15]:
list(df['question'])[:15]

['Как узнать условия кредитования?',
 'Могу ли я взять кредит в Сбербанке, если у меня уже есть кредиты?',
 'Как подать заявку на кредит в Сбербанке?',
 'Как оформить кредит в Сбербанк Онлайн?',
 'Какой срок рассмотрения заявки на кредит в Сбербанке?',
 'Как узнать статус рассмотрения моей заявки на кредит?',
 'Мне предварительно одобрили кредит. Что дальше?',
 'Как получить деньги по одобренной заявке?',
 'Как узнать дату и сумму платежа по кредиту?\n',
 'Как узнать, из чего состоит мой ежемесячный платеж по кредиту?',
 'Как узнать задолженность по кредиту?',
 'Как узнать, есть ли у меня задолженность по кредитам в Сбербанке?',
 'Как узнать, есть ли у меня непогашенные кредиты в Сбербанке?',
 'Как мне оформить досрочное погашение по кредиту?',
 'Как узнать, какие неустойки мне начислили, если я не погасил кредит вовремя?']